In [1]:
import numpy as np
import sqlalchemy
from pandas import DataFrame
from sqlalchemy.engine import URL
import pandas as pd
import plotly.express as px
import pyodbc

#pyodbc parameter for conections to database
driver = '{SQL Server}'
server = '10.10.20.37'
database = 'RISKPAN'
trust_connection = 'yes'

db_flavor = 'mssql+pyodbc'

#Conection String
conn_string = f'DRIVER={driver};'
conn_string += f'SERVER={server};'
conn_string += f'DATABASE={database};'
conn_string += f'TRUSTED_CONNECTION={trust_connection}'

In [2]:
#Sqlalchemy engine connection URL
conn_url = URL.create(db_flavor,query={"odbc_connect":conn_string})

##Create engine connection
engine = sqlalchemy.create_engine(conn_url)



In [8]:
# Read periodos_transcurridos_x_coshechas from sql to pandas.data frame
df_periodosbyvintages = pd.read_sql_table('periodos_transcurridos_x_cosecha', engine)

# Agrego campo para crear el campo acumulado
df_periodosbyvintages["Hipotecas_deemed_acumulado"] = 0
df_periodosbyvintages["Hipotecas_default_acumulado"] = 0
df_periodosbyvintages["personal_deemed_acumulado"] = 0
df_periodosbyvintages["personal_default_acumulado"] = 0

# Se lee el deemdefault de cada cosecha, se une al dataframe periodos y se genera un cumsum()
df_deemed_total: DataFrame = pd.DataFrame()
primera_vez = 1
vintage1 = df_periodosbyvintages['vintage'].min()
vintageN = df_periodosbyvintages['vintage'].max() + 1

# Se va obteniendo los deemeed defautls cosecha por cosecha porque si se hace el select para todas las
#cosechas la vista tarda demasiado tiempo. Una posibilidad es no usar una vista y calcular los deemede defaults
#usando un procedimiento o mejorar el performance de la vista
for vvintage in range(vintage1,vintageN):
    qry_deemed = """ select * from DEEMED_DEFAULT_POR_COSECHAS_HIPO where vintage = {} order by 2""".format(vvintage)
    df_deemed_defaults_vintage = pd.read_sql(qry_deemed, engine)
    if primera_vez:
        df_deemed_total = df_deemed_defaults_vintage
        primera_vez = 0
    else:
        #cada vintage leida se va apilando en un dataframe que las tendra todas
        df_deemed_total = df_deemed_total.append(df_deemed_defaults_vintage)


df_periodosbyvintages = pd.merge(df_periodosbyvintages, df_deemed_total, how='left', on=['vintage', 'period'])
df_periodosbyvintages.fillna(0, inplace=True)
df_periodosbyvintages['Hipotecas_deemed_acumulado'] = df_periodosbyvintages.groupby('vintage')['percentage_deemed_defaulted'].cumsum()


#Copio la estructura del Dataframe sin datos para usarlo en el calculo del deemed de todas las cosechas
dd_all_vintages = pd.DataFrame(data=None, columns=df_periodosbyvintages.columns)

#Paso los periodos del primer vintage el cual tiene todos los periodos
dd_all_vintages['period'] = df_periodosbyvintages[df_periodosbyvintages['vintage'] == vintage1].reset_index()['period']

#Se crea el vintage de todas las cosechas entre primer vintage y el ultimo vintage
dd_all_vintages['vintage'] = int(str(vintage1) + str(vintageN-1)) #creo la etiqueta que tiene desde hasta cosecha
dd_all_vintages.fillna(0, inplace=True)

#Calculo el deemed default de todas las cosechas
df_balance_def = df_periodosbyvintages.groupby('period')['balance_deemed_default'].sum().reset_index()
df_disbursement = df_periodosbyvintages.groupby('period')['disbursement'].sum().reset_index()
dd_all_vintages['percentage_deemed_defaulted'] = df_balance_def['balance_deemed_default'] / df_disbursement['disbursement']
dd_all_vintages.fillna(0, inplace=True)
dd_all_vintages['Hipotecas_deemed_acumulado'] = dd_all_vintages.groupby('vintage')['percentage_deemed_defaulted'].cumsum()

dd_all_vintages.fillna(0,inplace =True)

#Agrego el dataframe de toda las cosechas al dataframe final
df_periodosbyvintages = df_periodosbyvintages.append(dd_all_vintages).reset_index()

#Borro los dataframe que no se necestan
del df_deemed_defaults_vintage , df_deemed_total, dd_all_vintages,df_disbursement,df_balance_def



df_plotear = df_periodosbyvintages[['vintage','period','Hipotecas_deemed_acumulado']]
df_plotear.to_csv('data/Pan_Hipo_DeemDefault.csv')